In [39]:
import pandas as pd
import numpy as np
import pathlib
import win32com.client as win32

## Passo 01: Importação de dados e tranformação para o formato de DataFrame

In [3]:
df_email = pd.read_excel('C:/Users/Kimura/Desktop/Projetos/Projeto_01/Projeto_01/Bases_Dados/Emails.xlsx')
df_lojas = pd.read_csv('C:/Users/Kimura/Desktop/Projetos/Projeto_01/Projeto_01/Bases_Dados/Lojas.csv', encoding='latin1', sep=';')
df_vendas = pd.read_excel('C:/Users/Kimura/Desktop/Projetos/Projeto_01/Projeto_01/Bases_Dados/Vendas.xlsx')

In [4]:
df_email.head()

,Loja,Gerente,E-mail
0,Iguatemi Esplanada,Helena,pythonimpressionador+helena@gmail.com
1,Shopping Midway Mall,Alice,pythonimpressionador+alice@gmail.com
2,Norte Shopping,Laura,pythonimpressionador+laura@gmail.com
3,Shopping Iguatemi Fortaleza,Manuela,pythonimpressionador+manuela@gmail.com
4,Shopping União de Osasco,Valentina,pythonimpressionador+valentina@gmail.com


In [5]:
df_lojas.head()

,ID Loja,Loja
0,1,Iguatemi Esplanada
1,2,Shopping Midway Mall
2,3,Norte Shopping
3,4,Shopping Iguatemi Fortaleza
4,5,Shopping União de Osasco


In [6]:
df_vendas.head()

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final
0,1,2019-01-01,1,Sapato Estampa,1,358,358
1,1,2019-01-01,1,Camiseta,2,180,360
2,1,2019-01-01,1,Sapato Xadrez,1,368,368
3,2,2019-01-02,3,Relógio,3,200,600
4,2,2019-01-02,3,Chinelo Liso,1,71,71


## Passo 02: Criar uma tabela para cada loja

In [8]:
df_vendas_full = df_vendas.merge(df_lojas, on='ID Loja', how='left')
df_vendas_full.head()

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final,Loja
0,1,2019-01-01,1,Sapato Estampa,1,358,358,Iguatemi Esplanada
1,1,2019-01-01,1,Camiseta,2,180,360,Iguatemi Esplanada
2,1,2019-01-01,1,Sapato Xadrez,1,368,368,Iguatemi Esplanada
3,2,2019-01-02,3,Relógio,3,200,600,Norte Shopping
4,2,2019-01-02,3,Chinelo Liso,1,71,71,Norte Shopping


In [9]:
dict_loja = {}
for loja in df_lojas['Loja']:
    dict_loja[loja] = df_vendas_full.loc[df_vendas_full['Loja'] == loja, :]

### Passo 03: Definição do dia do indicador

In [11]:
dia_indicador = df_vendas_full['Data'].max()
dia_indicador

Timestamp('2019-12-26 00:00:00')

### Passo 04: Salvando os arquivos na pasta Backup


In [13]:
# Necessário importar a biblioteca Pathlib

caminho = pathlib.Path(r'Backup_Lojas')

arquivo_backup = caminho.iterdir()

lista_nomes_backup = [arquivo.name for arquivo in arquivo_backup]

for loja in dict_loja:
    if loja not in lista_nomes_backup:
        new_file = caminho / loja
        new_file.mkdir()
    nome_arquivo = '{}_{}_{}_{}.xlsx'.format(dia_indicador.day, dia_indicador.month,dia_indicador.year , loja)
    print(nome_arquivo)
    local_arquivo = caminho / loja / nome_arquivo

    dict_loja[loja].to_excel(local_arquivo)


26_12_2019_Iguatemi Esplanada.xlsx
26_12_2019_Shopping Midway Mall.xlsx
26_12_2019_Norte Shopping.xlsx
26_12_2019_Shopping Iguatemi Fortaleza.xlsx
26_12_2019_Shopping União de Osasco.xlsx
26_12_2019_Shopping Center Interlagos.xlsx
26_12_2019_Rio Mar Recife.xlsx
26_12_2019_Salvador Shopping.xlsx
26_12_2019_Rio Mar Shopping Fortaleza.xlsx
26_12_2019_Shopping Center Leste Aricanduva.xlsx
26_12_2019_Ribeirão Shopping.xlsx
26_12_2019_Shopping Morumbi.xlsx
26_12_2019_Parque Dom Pedro Shopping.xlsx
26_12_2019_Bourbon Shopping SP.xlsx
26_12_2019_Palladium Shopping Curitiba.xlsx
26_12_2019_Passei das Águas Shopping.xlsx
26_12_2019_Center Shopping Uberlândia.xlsx
26_12_2019_Shopping Recife.xlsx
26_12_2019_Shopping Vila Velha.xlsx
26_12_2019_Shopping SP Market.xlsx
26_12_2019_Shopping Eldorado.xlsx
26_12_2019_Shopping Ibirapuera.xlsx
26_12_2019_Novo Shopping Ribeirão Preto.xlsx
26_12_2019_Iguatemi Campinas.xlsx
26_12_2019_Shopping Barra.xlsx


### Passo 05: Cálculo de indicadores

In [15]:
# Faturamento:
loja1 = 'Norte Shopping'

vendas_loja_ano = dict_loja[loja1]
vendas_loja_dia = df_vendas_full.loc[df_vendas_full['Data'] == dia_indicador, :]

# Ano
faturamento_ano = vendas_loja_ano['Valor Final'].sum()
print(faturamento_ano)

#
faturamento_dia = vendas_loja_dia['Valor Final'].sum()
print(faturamento_dia)

1711968
21591


In [16]:
# Produtos

# Ano
qtde_produtos_ano = vendas_loja_ano['Produto'].unique()
print(len(qtde_produtos_ano))

# dia
qtde_produtos_dia = vendas_loja_dia['Produto'].unique()
print(len(qtde_produtos_dia))

120
35


In [17]:
# Ticket
# Precisamos considerar apenas os valores que vamos precisar para realizar os cálculos.

col_num_ano = vendas_loja_ano.select_dtypes(include=[np.number]).columns
vendas_ano = vendas_loja_ano.groupby('Código Venda')[col_num_ano].sum()
tkt_medio_ano = vendas_ano['Valor Final'].mean()

# Venda dia
col_num_dia = vendas_loja_dia.select_dtypes(include=[np.number]).columns
vendas_dia = vendas_loja_dia.groupby('Código Venda')[col_num_dia].sum()
tkt_medio_dia = vendas_dia['Valor Final'].mean()

print('Ticket médio ano: {:.2f}'.format(tkt_medio_ano))
print('Ticket médio dia: {:.2f}'.format(tkt_medio_dia))

Ticket médio ano: 784.59
Ticket médio dia: 938.74


In [18]:
#definição de metas

meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtdeprodutos_dia = 4
meta_qtdeprodutos_ano = 120
meta_ticketmedio_dia = 500
meta_ticketmedio_ano = 500

### Passo 06: Envio de e-mail para gerentes

In [47]:
outlook = win32.Dispatch('outlook.application')

mail = outlook.CreateItem(0)
mail.To = df_email.loc[df_email['Loja']==loja1 , 'E-mail'].values[0]
mail.CC = mail.BCC = ''
mail.Subject = ''
mail.Body =''

com_error: (-2147467260, 'Operação anulada', None, None)

In [ ]:
print(emails.loc[df_email['Loja']==loja1 , 'E-mail'].values[0])

In [41]:
df_email.head(5)

,Loja,Gerente,E-mail
0,Iguatemi Esplanada,Helena,pythonimpressionador+helena@gmail.com
1,Shopping Midway Mall,Alice,pythonimpressionador+alice@gmail.com
2,Norte Shopping,Laura,pythonimpressionador+laura@gmail.com
3,Shopping Iguatemi Fortaleza,Manuela,pythonimpressionador+manuela@gmail.com
4,Shopping União de Osasco,Valentina,pythonimpressionador+valentina@gmail.com
